## Install packages

In [47]:
from google.colab import files

In [51]:
%cd /content

/content


In [49]:
# clone the repo
!git clone https://github.com/nhvd3500111/ditto
%cd ditto/

# install requirements
!pip install -r requirements.txt

# if the above command does not work:
# !pip install transformers
# !pip install tensorboardX

%cd ..

import nltk
nltk.download('stopwords')

Cloning into 'ditto'...
remote: Enumerating objects: 303, done.
remote: Total 303 (delta 0), reused 0 (delta 0), pack-reused 303
Receiving objects: 100% (303/303), 26.88 MiB | 19.02 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Checking out files: 100% (129/129), done.
/content/ditto
  Using cached gensim-3.8.1-cp37-cp37m-manylinux1_x86_64.whl (24.2 MB)
  Using cached numpy-1.19.2-cp37-cp37m-manylinux2010_x86_64.whl (14.5 MB)
  Using cached scipy-1.3.2-cp37-cp37m-manylinux1_x86_64.whl (25.2 MB)
  Using cached sentencepiece-0.1.85-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
  Using cached spacy-3.1.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.4 MB)
ERROR: Could not find a version that satisfies the requirement torch==1.9.0+cu111 (from versions: 1.0.0, 1.0.1, 1.0.1.post2, 1.1.0, 1.2.0, 1.3.0, 1.3.1, 1.4.0, 1.5.0, 1.5.1, 1.6.0, 1.7.0, 1.7.1, 1.8.0, 1.8.1, 1.9.0, 1.9.1, 1.10.0, 1.10.1, 1.10.2, 1.11.0)
ERROR: No matching distribution found for torch==1.9.0+cu111
/cont

True

## Install fp16 optimization

In [3]:
!git clone https://github.com/NVIDIA/apex
%cd apex
!pip install -v --no-cache-dir ./
%cd ..

Cloning into 'apex'...
remote: Enumerating objects: 9746, done.
remote: Counting objects: 100% (817/817), done.
remote: Compressing objects: 100% (430/430), done.
remote: Total 9746 (delta 517), reused 585 (delta 368), pack-reused 8929
Receiving objects: 100% (9746/9746), 14.85 MiB | 19.29 MiB/s, done.
Resolving deltas: 100% (6628/6628), done.
/content/apex
Using pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
Value for scheme.platlib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.purelib does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/lib/python3.7/dist-packages
sysconfig: /usr/lib/python3.7/site-packages
Value for scheme.headers does not match. Please report this to <https://github.com/pypa/pip/issues/9617>
distutils: /usr/local/include/python3.7/UNKNOWN
sysconfig: 

In [4]:
# some issue with colab
!pip install --upgrade "urllib3==1.25.4" awscli

     |████████████████████████████████| 125 kB 5.0 MB/s 
     |████████████████████████████████| 3.9 MB 21.2 MB/s 
     |████████████████████████████████| 548 kB 52.5 MB/s 
     |████████████████████████████████| 79 kB 6.9 MB/s 
     |████████████████████████████████| 8.8 MB 32.8 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: rsa
    Found existing installation: rsa 4.8
    Uninstalling rsa-4.8:
      Successfully uninstalled rsa-4.8
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is inco

In [2]:
%cd ditto

/content/ditto


In [53]:
!pip install transformers
!pip install tensorboardX
!pip install jsonlines

In [4]:
# Step 1:
# train the wdc model with the save_model flag on. This is the GRU Ditto Model
# it will produce two files *_dev.pt and *_test.pt which are the 
# best checkpoints on the dev set and the test set.

!CUDA_VISIBLE_DEVICES=0 python train_ditto.py \
  --task Structured/Fodors-Zagats \
  --batch_size 32 \
  --max_len 128 \
  --lr 3e-5 \
  --n_epochs 20 \
  --finetuning \
  --lm roberta \
  --fp16 \
  --save_model \
  --da drop_col 


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  Futu

## Running the matcher

In [60]:
# Step 2: run the matcher


!CUDA_VISIBLE_DEVICES=0 python matcher.py \
  --task Structured/Fodors-Zagats \
  --input_path data/er_magellan/Structured/Fodors-Zagats/test.txt \
  --output_path output/output_small_1.jsonl \
  --lm roberta \
  --max_len 128 \
  --use_gpu \
  --fp16 \
  --checkpoint_path checkpoints/ \
  --neural gru

files.download('F1_SCORES.xlsx')

Traceback (most recent call last):
  File "matcher.py", line 22, in <module>
    from ditto_light.ditto import evaluate, DittoModel
  File "/content/ditto/ditto_light/ditto.py", line 9, in <module>
    import sklearn.metrics as metrics
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/__init__.py", line 67, in <module>
    from ._regression import explained_variance_score
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 31, in <module>
    from .._loss.glm_distribution import TweedieDistribution
  File "<frozen importlib._bootstrap>", line 983, in _find_and_load
  File "<frozen importlib._bootstrap>", line 967, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 670, in _load_unlocked
  File "<frozen importlib._bootstrap>", line 589, in module_from_spec
  File "<frozen importlib._bootstrap>", line 553, in _init_module_attrs
KeyboardInterrupt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>